In [ ]:
import duckdb
import polars as pl


pl.Config.set_tbl_rows(100)

In [ ]:
dataset_name = 'airbnb'

In [ ]:
file_path = f'../data/{dataset_name}/{dataset_name}'

In [ ]:
con = duckdb.connect(database = ':memory:', read_only=False)

In [ ]:
con.execute(f"""CREATE OR REPLACE TABLE airbnb AS SELECT * FROM '{file_path}.csv'""")

In [ ]:
con.sql('FROM airbnb limit 10')

In [ ]:
#Zipcode data is poor (nulls and incomplete), 
#latitude and longtitude seems good. 
#room_type is good
#accomodates is good, but is bigint and should be int
#bedrooms has nulls, is a double and need to be int
#review_scores_values has nulls and should be int
#price seems good but has very low ones.
#review_scores value seems good but has nulls and should be int
con.sql('SUMMARIZE airbnb').show(max_width=250)

In [ ]:
#Lots of weird things going on with the zipcodes
con.sql('SELECT length(zipcode), count(*) from airbnb group by length(zipcode)').show()

#Probably best to only use valid zipcodes
con.sql("select count(*) from airbnb WHERE zipcode ~ '^[0-9]{4} [A-Z]{2}$'").show()

In [ ]:
con.sql('SELECT distinct room_type from airbnb')

In [ ]:
con.sql('select * from airbnb where price >= 15 and price <= 20')

In [ ]:
#Cant find a way to show all rows in duckdb. table gets truncated.
#Cant transform to a polars dataframe for some reason. 
# df = con.sql('select distinct review_scores_value from airbnb').pl()


review_scores_value = pl.read_csv(file_path+'.csv').select('review_scores_value').unique()
display(review_scores_value)

In [ ]:
#We will end up with this table
con.sql("""
    CREATE OR REPLACE TABLE airbnb_valid_zipcodes AS 
    SELECT * FROM airbnb
    WHERE zipcode ~ '^[0-9]{4} [A-Z]{2}$'
""")

In [ ]:
con.sql('select count(*) from airbnb_valid_zipcodes where zipcode is null')

In [ ]:
con.sql('SUMMARIZE airbnb_valid_zipcodes').show(max_width=250)